Importing

In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings('ignore')

MAIN_DIR = "D:\\College_Semesters\\7th Semester\\F. Final\\Codes\\Datasets\\Initial_AugmentedDataset"
SEED = 40
os.listdir(MAIN_DIR)

['10_Shikara',
 '11_Kapitta',
 '12_Katakamukhaha',
 '13_Soochi',
 '14_Chandrakala',
 '15_Padmakosha',
 '16_Sarpasheeshaha',
 '17_Mrughasheeha',
 '18_Sihamukhaha',
 '19_Kangoola',
 '1_Pataka',
 '20_Alapadma',
 '21_Chatura',
 '22_Bramhara',
 '23_Hamsasya',
 '24_Hamsapashakaha',
 '25_Shandamshu',
 '26_Samadamaso',
 '27_Mukula',
 '28_Tamrachooda',
 '29_Trishoola',
 '2_Tripataka',
 '3_Ardhapataka',
 '4_Kartarimukhaha',
 '5_Mayura',
 '6_Ardhachandra',
 '7_Arala',
 '8_Shukatundakaha',
 '9_Mushti']

In [4]:
def load_images(folder):
    imgs = []
    target = 0
    labels = []
    for i in os.listdir(folder):
        subdir = os.path.join(folder, i)
        for j in os.listdir(subdir):
            img_dir = os.path.join(subdir,j)
            try:
                img = cv2.imread(img_dir)
                if img.shape[-1] == 1:
                    # convert grayscale image to RGB
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                img = cv2.resize(img,(224,224))
                imgs.append(img)
                labels.append(target)
            except:
                continue
        target += 1
    
    imgs = np.array(imgs)
    labels = np.array(labels)

    return imgs, labels

In [5]:
data, labels = load_images(MAIN_DIR)

In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Resize images
new_width, new_height = 128, 128  # Adjust according to your requirements
resized_data = [cv2.resize(img, (new_width, new_height)) for img in data]
norm_data = np.array(resized_data) / 255.0

# Encode labels
encoder = LabelEncoder()
encoder.fit(labels)
y = encoder.transform(labels)
y_one_hot = to_categorical(y, num_classes=29)

In [7]:
# Split the data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Print the shapes of the split datasets
print("Train Data Shape:", train_data.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Data Shape:", test_data.shape)
print("Test Labels Shape:", test_labels.shape)

Train Data Shape: (3480, 224, 224, 3)
Train Labels Shape: (3480,)
Test Data Shape: (870, 224, 224, 3)
Test Labels Shape: (870,)


In [8]:
from tensorflow.keras.utils import to_categorical

# Convert target labels to one-hot encoded format
train_labels = to_categorical(train_labels, num_classes=29)
test_labels = to_categorical(test_labels, num_classes=29)

In [9]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Concatenate, GlobalAveragePooling2D, Dense
# from tensorflow.keras.callbacks import EarlyStopping

# # Assuming new_width and new_height are defined
# input_shape = (224, 224, 3)

# def dense_block(x, num_blocks, growth_rate):
#     for _ in range(num_blocks):
#         y = BatchNormalization()(x)
#         y = ReLU()(y)
#         y = Conv2D(growth_rate, kernel_size=(3, 3), padding='same')(y)
#         x = Concatenate()([x, y])
#     return x

# model = Sequential()

# # Initial Convolution layer
# model.add(Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', input_shape=input_shape))
# model.add(BatchNormalization())
# model.add(ReLU())

# # First dense block
# x = model.layers[-1].output  # Save the output of the previous layer
# model.add(dense_block(x, num_blocks=6, growth_rate=32))

# # Transition layer
# model.add(Conv2D(128, kernel_size=(1, 1), strides=(2, 2), padding='same'))
# model.add(BatchNormalization())
# model.add(ReLU())

# # Second dense block
# x = model.layers[-1].output
# model.add(dense_block(x, num_blocks=12, growth_rate=32))

# # Transition layer
# model.add(Conv2D(256, kernel_size=(1, 1), strides=(2, 2), padding='same'))
# model.add(BatchNormalization())
# model.add(ReLU())

# # Third dense block
# x = model.layers[-1].output
# model.add(dense_block(x, num_blocks=24, growth_rate=32))

# # Transition layer
# model.add(Conv2D(512, kernel_size=(1, 1), strides=(2, 2), padding='same'))
# model.add(BatchNormalization())
# model.add(ReLU())

# # Global Average Pooling
# model.add(GlobalAveragePooling2D())

# # Fully connected layer
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(29, activation='softmax'))  # Adjust the number of units based on your number of classes (29)

# # Compile the model
# model.compile(optimizer='adam',
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Define early stopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, min_delta=0.001)

# # Fit the model with early stopping
# history = model.fit(train_data, train_labels, epochs=50, batch_size=32, 
#                     validation_data=(test_data, test_labels),
#                     callbacks=[early_stopping])

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping

def dense_block(x, num_blocks, growth_rate):
    for _ in range(num_blocks):
        y = BatchNormalization()(x)
        y = ReLU()(y)
        y = Conv2D(growth_rate, kernel_size=(3, 3), padding='same')(y)
        x = Concatenate()([x, y])
    return x

def transition_layer(x, num_filters):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(num_filters, kernel_size=(1, 1), strides=(2, 2), padding='same')(x)
    return x

input_shape = (224, 224, 3)

# Input layer
inputs = Input(shape=input_shape)

# Initial Convolution layer
x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)

# Dense Blocks and Transition Layers
x = dense_block(x, num_blocks=6, growth_rate=32)
x = transition_layer(x, num_filters=128)

x = dense_block(x, num_blocks=12, growth_rate=32)
x = transition_layer(x, num_filters=256)

x = dense_block(x, num_blocks=24, growth_rate=32)
x = transition_layer(x, num_filters=512)

# Final dense block without a following transition layer
x = dense_block(x, num_blocks=16, growth_rate=32)

# Global Average Pooling and Fully connected layers
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(29, activation='softmax')(x)  # Adjust the number of units based on your number of classes (29)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, min_delta=0.001)

# Fit the model with early stopping
# Replace 'train_data', 'train_labels', 'test_data', 'test_labels' with your actual data
history = model.fit(train_data, train_labels, epochs=50, batch_size=32, 
                    validation_data=(test_data, test_labels),
                    callbacks=[early_stopping])

Epoch 1/50


: 

In [ ]:
model.save("DenseNet\\DenseNet_New_1.h5")

In [ ]:
# Evaluate the model on the test data
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test Loss:', test_loss)

accuracy = np.mean(predicted_labels == true_labels)
print('Accuracy:', accuracy*100)

In [ ]:
from sklearn.metrics import classification_report

# Make predictions on the test data
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels, axis=1)

# Print the classification report
print(classification_report(true_labels, predicted_labels))

In [ ]:
model.summary()

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()